# Run once

In [3]:
%matplotlib qt5
import matplotlib.pyplot as plt  
import matplotlib.dates as mdates
import numpy as np
import visa
import time
import datetime
import csv
import os
import sys
# Add path
sys.path.append('..\Instrument Driver')
sys.path.append('..\Helper')

# instrument driver
import SR830

class Plotter:
    """
    Description:
    """
    def __init__(self, xlabel='xlabel()', ylabel='ylabel()', title='title', plotStyle='b-'):
        self.fig, self.ax = plt.subplots(1,1)
        self.line, = self.ax.plot([], [], plotStyle) 
        self.fig.suptitle(title)
        self.fig.autofmt_xdate()
        
        self.ax.set_xlabel(xlabel)
        self.ax.set_ylabel(ylabel)
        self.ax.xaxis.set_major_formatter(mdates.DateFormatter("%H:%M"))
        self.ax.axes.axes.grid(1) #This is for the axes.ax
        
    def update(self, x, y):
        self.line.set_xdata(x)
        self.line.set_ydata(y)

        self.ax.relim()
        self.ax.autoscale()
        
        self.fig.canvas.draw() 
        self.fig.canvas.flush_events()
        
def getTemperature(R_volt):
    R = R_volt / (0.01 / 10e6)  # R: resistance of thermometer
    if R > 3590:
        Temp = 377*((1/(R-1500))+8.22e-6)**0.86296
    else:
        Temp = 4031*((1/(R-997.7))+4.54e-5)**1.15512

    return Temp

def configure_lockin(lockin):
    """configure lockin parameters
       Arg: 
           lockin: instance of lockin instrument driver
    """
    lockin.setTrig('sine')
    lockin.setSource( 'int')
    lockin.setHarm(1)
    lockin.setAmpl(0.01)  # Volts
    lockin.setFreq(13) # Hz

    lockin.setShield('ground')
    lockin.setCoupling('DC')
    lockin.setInputConfig('A-B')

    lockin.setTimeConst('1 s')
    lockin.setSlope(12)

    lockin.setSens('50 uV/pA')

    lockin.setReserve('normal')
    lockin.setFilter('both')


In [ ]:
# initialize SR830 - (Lock-in Amplifier)
GPIB_add = 13
lockin = SR830.SR830(GPIB_add)
configure_lockin(lockin)

# Measure FAA temperature continuously

In [4]:
###### parameters  ################
save_path = r'Z:\User\Vito'
fileName = 'ADR1_FFA_temper_' + datetime.datetime.strftime(datetime.datetime.now(), '%y%m%d_%H%M%S')
sleep_interval = 3  # in sec
####################################

# initialize SR830 - (Lock-in Amplifier)
GPIB_add = 13
lockin = SR830.SR830(GPIB_add)
configure_lockin(lockin)

# Initialize plot
plotobj = Plotter(ylabel = 'Temperature (K)', xlabel='Time', title = 'ADR1 Cold Finger Temperature', plotStyle='b-')

# Do loop
time_arr, temper_arr = [], []

fullFileName = os.path.join(save_path, fileName + '.csv') 

with open(fullFileName, 'w') as f:
    writer = csv.writer(f)
    
    while True:        
        # measure time and resistance
        curr_time = datetime.datetime.now()
        curr_temperature = np.abs(getTemperature(float(lockin.readValue('r'))))

        # update time and temperature array
        time_arr.append(curr_time)
        temper_arr.append(curr_temperature)

        # update plot
        plotobj.update(time_arr, temper_arr)
        
        # save data in csv
        row = [curr_time.strftime('%m/%d/%Y %H:%M:%S'), curr_temperature]
        writer.writerow(row)
        
        # sleep
        time.sleep(sleep_interval)


Reference trigger set to sine 0 crossing! 
Source set to internal!
Detection harmonic set to 1! 
Amplitude set to 0.01! 
Frequency set to 13! 
Input shield grounding set to ground!
Input coupling set to DC!
Input configuration (current gain) set to A-B! 
Slope set to 12 dB/oct!
Mode set to normal reserve!
Both filters in!


KeyboardInterrupt: 

In [6]:
import importlib
importlib.reload(SR830)

<module 'SR830' from '..\\Instrument Driver\\SR830.py'>